In [13]:
import pandas as pd
import json
import numpy as np
import ast
from pandas.io.json import json_normalize #package for flattening json in pandas df
from tqdm import tqdm
pd.set_option('display.max_colwidth',1000)

In [2]:
import mmap

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

### sample the data so that it looks like this:

TEXT | product categories | rating given

In [3]:
root = "/media/felipe/SAMSUNG/AmazonReviews/"

In [4]:
reviews_df = pd.read_json(root+"/sample_reviews_Books_5.json",lines=True)

In [5]:
reviews_df.head(1)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0441019420,"[0, 0]",4,i'm a big Patricia Briggs fan. i love her stor...,"11 7, 2010",A18O0N0QI055FU,"Summer D. Olson ""fredomoftruth""",great book,1289088000


In [6]:
reviews_df = reviews_df[['asin','overall','reviewText']]
reviews_df.index = reviews_df['asin']
reviews_df.drop(['asin'],axis=1,inplace=True)

In [7]:
reviews_df['categories'] = np.nan

In [8]:
reviews_df.head()

,overall,reviewText,categories
asin,,,
0441019420,4,i'm a big Patricia Briggs fan. i love her stor...,NaN
0316051632,5,"The Disappearing Spoon by Sam KeanLittle, Brow...",NaN
038573901X,2,"I enjoy a great romance novel now and again, a...",NaN
006144295X,1,I've read all of Kimberla Lawson-Roby novels. ...,NaN
0316097543,3,This book almost made it. The jump from starv...,NaN


In [9]:
with open(root+"/sample_metadata.json") as f:
    for line in tqdm(f, total=get_num_lines(root+"/sample_metadata.json")):
        json_data = ast.literal_eval(line)
        other_df = json_normalize(json_data)
        other_df['asin'] = other_df['asin'].astype('object')
        other_df.index = other_df['asin']
        other_df.drop(['asin'],axis=1,inplace=True)
               
        if not 'categories' in other_df.columns.values:
            other_df['categories'] = ''
           
        reviews_df.update(other_df)
        

100%|██████████| 5000000/5000000 [5:16:04<00:00, 263.65it/s]  


In [11]:
reviews_df.head()

,overall,reviewText,categories
asin,,,
0441019420,4,i'm a big Patricia Briggs fan. i love her stor...,NaN
0316051632,5,"The Disappearing Spoon by Sam KeanLittle, Brow...",NaN
038573901X,2,"I enjoy a great romance novel now and again, a...",[[Books]]
006144295X,1,I've read all of Kimberla Lawson-Roby novels. ...,NaN
0316097543,3,This book almost made it. The jump from starv...,NaN


In [14]:
reviews_df[reviews_df['categories'].notnull()]

,overall,reviewText,categories
asin,,,
038573901X,2,"I enjoy a great romance novel now and again, and I like it when there are relationships in most of the YA books that I read. But the key word for me is romance, not just hormones and sex. I felt like Ari, who is too good to be true in many aspects, was a little too fixated on boys in this book between her long-standing crush on her brother-in-law, her ennui over her ex-boyfriend, her interactions with Del, and her relationship with Blake. While I remember being 17 and dealing with those crazy teenage hormones, that in and of itself isn't altogether that interesting to read about. At least not the way it's done here.There are a few conversations between Ari and Blake about what they want in the future, but overall I felt like most of the book was about her wanting a boyfriend and her boy troubles, with the occasional friend, family, and SAT drama tossed in. I'd be okay with a book that was all about a relationship if there was more joy or depth or caring to it (see Sixteenth Summer ...",[[Books]]
0375868275,5,Obvisously written for youth reading but enjoyed by this senior citizen. Includes lessons in human nature and environmental issues in light hearted satire.,[[Books]]
1477848169,3,i wanted to read this given the Amazon award had won and now i have mixed feelings whether i'd read another winner of this aware. i'm sorry i just found the voice too feminine for a straight guy. i kept laughing at parts thinking when is this guy coming out of the closet. i did like his dry humour though. there were some typos in there that surprised me because I thought Amazon would've done a thorough editing job or is this up to the author? as a paranormal fantasy it was okay/ the plot sort of got a little muddy 2nd half so i found myself starting to skim. i did enjoy the girlfriend's character very much -- she was my fave.,[[Books]]
B00JYOM60K,5,"This is book 4 and you *really* need to read 1-3 first! What I Want wraps up the Stranded series. It ties up the story, while leaving room for another story at a later date.I adore Drake and Ashley. In What I want, we get to see into Drake's head, experience what he was thinking and feeling back in the cabin and since. I love seeing how Ashley is realizing that what she wants may not be what is best for her. It's a tough place to be in. Noelle Stevens did a great job portraying how vulnerable and strong Ashley is. Emily is also more involved in the story and is a much needed voice of reason.","[[Books, Literature & Fiction, Genre Fiction, Family Saga], [Books, Romance, Anthologies], [Books, Romance, New Adult & College], [Kindle Store, Kindle eBooks, Literature & Fiction, Genre Fiction, Sagas], [Kindle Store, Kindle eBooks, Romance, Collections & Anthologies], [Kindle Store, Kindle eBooks, Romance, New Adult & College]]"
0465027687,1,"Weigel's entire premise rests on the idea that we are in some sort of 500 year change in the world, and that the Church in order to be relevant, needs to properly identify this change so that we can reestablish our evangelical roots. Unfortunately, Weigel has obviously not spent much time with any of the writings of the Church prior to Vatican II or any pope prior to John Paul II. I have no problem with either Vatican II, or Blessed John Paul II, but there WAS a church prior to both of them, and Weigel would be well served to explore it.The real issue is one that is underlying and one that Weigel seems to hint at, but will not engage with. It is the issue of what is the true nature of the Church and what is her level of necessity when it comes to salvation. Weigel, unfortunately, seems to subscribe to the general consensus among the laity and all too many of the clerics in the Church that there is nothing truly special about the Bride of Christ, and that one is just as likely ...",[[Books]]
149299054X,5,Bought it. Could not put it down. Finished it in one day. The story moved along at a furious pace on various fronts.,[

In [ ]:
sample_metadata_df = pd.read_json(root+"/sample_metadata.json",lines=True)